In [19]:
import os


os.listdir('/content/')


['.config',
 'combined_eeg_data_with_labels.csv',
 'normal.csv',
 'complex.csv',
 'video.csv',
 '.ipynb_checkpoints',
 'electro.csv',
 'Validation',
 'sample_data']

In [20]:
import pandas as pd

# Load each CSV file (replace the names with your actual file names)
df_complex_partial = pd.read_csv('/content/complex.csv')
df_electrographic = pd.read_csv('/content/electro.csv')
df_video_detected = pd.read_csv('/content/video.csv')
df_normal = pd.read_csv('/content/normal.csv')

df_complex_partial['label'] = 1  # Complex Partial Seizures
df_electrographic['label'] = 2   # Electrographic Seizures
df_video_detected['label'] = 3   # Video-detected Seizures
df_normal['label'] = 0           # Normal Data

# Concatenate
df_all = pd.concat([df_complex_partial, df_electrographic, df_video_detected, df_normal], ignore_index=True)

X = df_all.drop(columns=['label'])  # Format x
y = df_all['label']  # Labels

df_all.to_csv('/content/combined_eeg_data_with_labels.csv', index=False)



In [21]:
X['file_name'] = X['file_name'].str.slice(0, -4) #remove .npy thing

In [22]:
from sklearn.preprocessing import StandardScaler

#Drop the 'file_name' column from X
X_data = X.drop(columns=['file_name'])

# Standardize/Normalize the features
scaler = StandardScaler()  # standardizes the data (mean=0, std=1)

# Fit the scaler and transform the data
X_scaled = scaler.fit_transform(X_data)


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Step 4: Train a classifier, for example, RandomForest
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Step 5: Predict and evaluate
y_pred = clf.predict(X_test)
print(f"Accuracy for the model using train: {accuracy_score(y_test, y_pred)}")


Accuracy for the model using train: 0.7890291398808127


In [24]:
print(f'The number of parameters is {X_train.shape[1] - 2}')

The number of parameters is 8


In [25]:
#here validation stuff
vdf_complex_partial = pd.read_csv('/content/Validation/complex.csv')
vdf_electrographic = pd.read_csv('/content/Validation/electro.csv')
vdf_video_detected = pd.read_csv('/content/Validation/video.csv')
vdf_normal = pd.read_csv('/content/Validation/normal.csv')

vdf_complex_partial['label'] = 1  # Complex Partial Seizures
vdf_electrographic['label'] = 2   # Electrographic Seizures
vdf_video_detected['label'] = 3   # Video-detected Seizures
vdf_normal['label'] = 0           # Normal Data

# Concatenate
df_all = pd.concat([vdf_complex_partial, vdf_electrographic, vdf_video_detected, vdf_normal], ignore_index=True)
vX = df_all.drop(columns=['label'])  # Format x
vX['file_name'] = X['file_name'].str.slice(0, -4) #remove .npy thing

vy = df_all['label']  # Labels


In [26]:
vX.dropna()

,file_name,channel,frequency,fft_magnitude,Dominant Frequency,Spectral Power,Spectral Entropy,Band Power Ratios (alpha/theta),Spectral Edge Frequency (SEF),Mean Frequency,Frequency Variability
0,,1,5,0.672749,5,1.350000e-05,3.716145,0.474621,33.203125,29.598334,36.623937
1,,2,5,0.848848,5,1.540000e-05,3.707717,0.227569,44.921875,31.880390,35.731150
2,,3,5,0.697533,5,4.870000e-06,3.496839,0.122334,42.968750,28.298564,34.705512
3,,4,5,0.485256,5,4.160000e-06,3.795937,0.223071,46.875000,26.891814,27.110089
4,,5,5,0.708606,5,4.360000e-06,3.024952,0.408245,27.343750,21.813564,28.190817
...,...,...,...,...,...,...,...,...,...,...,...
26652,,15,5,0.197797,5,3.750000e-07,3.035215,0.550551,15.625000,37.887941,56.919479
26653,,16,2,0.234002,2,4.710000e-07,2.669355,0.275536,11.718750,19.825148,34.754488
26654,,17,2,0.421779,2,2.180000e-06,2.648394,0.074881,13.671875,27.103029,45.718844
26655,,18,9,0.489859,9,5.000000e-06,2.812524,0.375545,17.578125,20.400630,35.679583


In [29]:
print(vX.isnull().sum())  # Check for missing values
print(vX.dtypes)          # Check data types


file_name                          0
channel                            0
frequency                          0
fft_magnitude                      0
Dominant Frequency                 0
Spectral Power                     0
Spectral Entropy                   0
Band Power Ratios (alpha/theta)    0
Spectral Edge Frequency (SEF)      0
Mean Frequency                     0
Frequency Variability              0
dtype: int64
file_name                           object
channel                              int64
frequency                            int64
fft_magnitude                      float64
Dominant Frequency                   int64
Spectral Power                     float64
Spectral Entropy                   float64
Band Power Ratios (alpha/theta)    float64
Spectral Edge Frequency (SEF)      float64
Mean Frequency                     float64
Frequency Variability              float64
dtype: object


In [30]:
vX = vX.drop(columns=['file_name'])
vy_pred = clf.predict(vX)
print(f"Accuracy for the model using validation: {accuracy_score(vy, vy_pred)}")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Accuracy for the model using validation: 0.3101249202836028


In [31]:
#now for test

In [34]:
test = pd.read_csv('/content/Untitled spreadsheet - test_data.csv')
testy_pred = clf.predict(test)
# Create a DataFrame with the predictions
predictions_df = pd.DataFrame(testy_pred, columns=['Prediction'])
# Save the DataFrame to a CSV file
predictions_df.to_csv('predictions.csv', index=False)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
